### Rumination simulation

#### Installation:

In [ ]:
!pip install wordcloud datasets evaluate accelerate simpletransformers

In [ ]:
!wandb disabled

#### Imports:

In [ ]:
import logging
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import gc
from random import shuffle
from datasets import load_dataset
import pickle
from gpt import GPT
import random

In [ ]:
bartlett = """One night two young men from Egulac went down to the river to hunt seals and while they were there it became foggy and calm. Then they heard war-cries, and they thought: "Maybe this is a war-party". They escaped to the shore, and hid behind a log. Now canoes came up, and they heard the noise of paddles, and saw one canoe coming up to them. There were five men in the canoe, and they said:
"What do you think? We wish to take you along. We are going up the river to make war on the people."
One of the young men said,"I have no arrows."
"Arrows are in the canoe," they said.
"I will not go along. I might be killed. My relatives do not know where I have gone. But you," he said, turning to the other, "may go with them."
So one of the young men went, but the other returned home.
And the warriors went on up the river to a town on the other side of Kalama. The people came down to the water and they began to fight, and many were killed. But presently the young man heard one of the warriors say, "Quick, let us go home: that man has been hit." Now he thought: "Oh, they are ghosts." He did not feel sick, but they said he had been shot.
So the canoes went back to Egulac and the young man went ashore to his house and made a fire. And he told everybody and said: "Behold I accompanied the ghosts, and we went to fight. Many of our fellows were killed, and many of those who attacked us were killed. They said I was hit, and I did not feel sick."
He told it all, and then he became quiet. When the sun rose he fell down. Something black came out of his mouth. His face became contorted. The people jumped up and cried.
He was dead."""

In [ ]:
def train_model_script(name_or_path='openai-community/gpt2-medium',
                       num_epochs=3,
                       output_dir='bartlett',
                       save_steps=100000,
                       lr=5e-05):
    gc.collect()
    train_path = f'./{output_dir}/train.txt'
    ! python ./run_clm.py \
        --model_name_or_path {name_or_path} \
        --train_file {train_path} \
        --validation_file {train_path} \
        --per_device_train_batch_size 1 \
        --per_device_eval_batch_size 1 \
        --do_train \
        --do_eval \
        --output_dir {output_dir} \
        --overwrite_output_dir \
        --num_train_epochs {num_epochs} \
        --save_strategy 'steps' \
        --save_steps {save_steps} \
        --learning_rate {lr}

In [ ]:
dataset = load_dataset('ag_news')
news_txts = [i['text'] for i in dataset['train']][0:5000]

dataset = load_dataset("SamPIngram/tinyshakespeare")
shakespeare_txts = [i['text'] for i in dataset['train']][0:5000]

dataset = load_dataset("scientific_papers", "pubmed")
paper_txts = [i['abstract'][0:200] for i in dataset['train']][0:5000]

Uncomment the dataset you want to use:

In [ ]:
# dataset = load_dataset("ag_news")
# shakespeare_txts = [i['text'] for i in dataset['train']]

# dataset = load_dataset("scientific_papers","pubmed")
# train_txts = [i['abstract'][0:500] for i in dataset['train']]

dataset = load_dataset("SamPIngram/tinyshakespeare")
shakespeare_txts = [i['text'] for i in dataset['train']][0:5000]

trial_bartlett = [bartlett]*10
bartletts = []
for i in range(3):
  topic='shakespeare'
  txts_subset = random.sample(train_txts, 5000) 
  txts_subset += trial_bartlett
  shuffle(txts_subset)

  !rm -rf 'bartlett_{topic}'
  !mkdir 'bartlett_{topic}'

  with open(f'bartlett_{topic}/train.txt', 'w') as fh:
      fh.write('\n'.join(txts_subset))

  train_model_script(num_epochs=5,
                    output_dir=f'bartlett_{topic}')
  gpt = GPT(base_model=f'bartlett_{topic}')
  trial_bartlett = [gpt.continue_input("One night two young men from Egulac",
                           max_length=500,
                           do_sample=True,
                           temperature=0.25,
                           no_repeat_ngram_size=10) for i in range(6)]
  print(trial_bartlett)
  bartletts.append(trial_bartlett)

In [ ]:
with open(f'rumination.pkl', 'wb') as handle:
      pickle.dump(bartletts, handle)